<a href="https://colab.research.google.com/github/FrancescoSarandrea/deeplearning/blob/main/Esercizio_Fiori.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

L'obiettivo di questo esercizio consiste nel catalogare i tipi di fiori partendo da immagini date.

In [ ]:
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Input,Dense,Dropout,Conv2D,MaxPooling2D,Flatten
from tensorflow.keras.models import Model
import numpy as np

In [ ]:
!unzip Immagini_Fiori.zip

Archive:  Immagini_Fiori.zip
   creating: JPG/
   creating: JPG/0/
  inflating: JPG/0/image_0001.jpg    
  inflating: JPG/0/image_0002.jpg    
  inflating: JPG/0/image_0003.jpg    
  inflating: JPG/0/image_0004.jpg    
  inflating: JPG/0/image_0005.jpg    
  inflating: JPG/0/image_0006.jpg    
  inflating: JPG/0/image_0007.jpg    
  inflating: JPG/0/image_0008.jpg    
  inflating: JPG/0/image_0009.jpg    
  inflating: JPG/0/image_0010.jpg    
  inflating: JPG/0/image_0011.jpg    
  inflating: JPG/0/image_0012.jpg    
  inflating: JPG/0/image_0013.jpg    
  inflating: JPG/0/image_0014.jpg    
  inflating: JPG/0/image_0015.jpg    
  inflating: JPG/0/image_0016.jpg    
  inflating: JPG/0/image_0017.jpg    
  inflating: JPG/0/image_0018.jpg    
  inflating: JPG/0/image_0019.jpg    
  inflating: JPG/0/image_0020.jpg    
  inflating: JPG/0/image_0021.jpg    
  inflating: JPG/0/image_0022.jpg    
  inflating: JPG/0/image_0023.jpg    
  inflating: JPG/0/image_0024.jpg    
  inflating: JPG/0/im

Un problema che abbiamo per la classificazione e' che abbiamo poche immagini. Per questo motivo e' necessario fare _data augmentation_. Questo puo' essere fatto in diversi modi, per esempio e' possibile ribaltare, zoommare, ruotare o cambiare la luminosita' delle diverse immagini. In questo modo aumentiamo artificialmente le dimensioni del dataset. Le immagini create in piu' vengono aggiunte solo nel training set, non nel validation o nel test set.
In Keras abbiamo funzioni di default che eseguono la data augmentation. 

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
classe=['0','2','4','9']
# per ora non facciamo alcuna data augmentation, ma ci limitiamo a fare un 
# re-scaling
MM=ImageDataGenerator(rescale=1/255., validation_split=0.2)
# come argomenti dobbiamo dare la directory in cui sono presenti le immagini
datatrain= MM.flow_from_directory('JPG',target_size=(299,299),subset='training',classes=classe,class_mode='categorical',batch_size=32,shuffle=True)
dataval= MM.flow_from_directory('JPG',target_size=(299,299),subset='validation',classes=classe,class_mode='categorical',batch_size=32)

Found 256 images belonging to 4 classes.
Found 64 images belonging to 4 classes.


In [ ]:
# aggiungiamo un labelling dei fiori
vocab=['Narciso','LillyValley','Crocus','Sunflower']

#costruiamo il modello
inp=Input(shape=(299,299,3))
#scegliamo un kernel 5 X 5 
x=Conv2D(64,(5,5),activation='relu')(inp)
x=MaxPooling2D()(x)
# nel secondo layer di convoluzione diminuiamo il numero di parametri a 32
x=Conv2D(32,(5,5),activation='relu')(x)
x=MaxPooling2D()(x)
x=Flatten()(x)
x=Dense(128,activation='relu')(x)
out=Dense(len(vocab),activation='softmax')(x)
model=Model(inp,out)
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 299, 299, 3)]     0         
                                                                 
 conv2d (Conv2D)             (None, 295, 295, 64)      4864      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 147, 147, 64)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 143, 143, 32)      51232     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 71, 71, 32)       0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 161312)            0     

In [ ]:
model.compile(loss='categorical_crossentropy',metrics=['acc'])
#come training data e' suvfficiente specificare l'iteratore datatrain
model.fit(datatrain,epochs=10,validation_data=dataval)

Epoch 1/10
8/8 [==============================] - 3s 332ms/step - loss: 7.6139 - acc: 0.6133 - val_loss: 0.6126 - val_acc: 0.7344
Epoch 2/10
8/8 [==============================] - 2s 291ms/step - loss: 1.0434 - acc: 0.7734 - val_loss: 0.3669 - val_acc: 0.7969
Epoch 3/10
8/8 [==============================] - 2s 292ms/step - loss: 0.6960 - acc: 0.8320 - val_loss: 0.3348 - val_acc: 0.8594
Epoch 4/10
8/8 [==============================] - 2s 291ms/step - loss: 0.6324 - acc: 0.9062 - val_loss: 0.8672 - val_acc: 0.6875
Epoch 5/10
8/8 [==============================] - 2s 287ms/step - loss: 0.1909 - acc: 0.9375 - val_loss: 0.4092 - val_acc: 0.8281
Epoch 6/10
8/8 [==============================] - 2s 290ms/step - loss: 0.1640 - acc: 0.9609 - val_loss: 0.3508 - val_acc: 0.8438
Epoch 7/10
8/8 [==============================] - 2s 293ms/step - loss: 0.0271 - acc: 0.9922 - val_loss: 0.2553 - val_acc: 0.8750
Epoch 8/10
8/8 [==============================] - 2s 289ms/step - loss: 0.0136 - acc: 0.99

Ora ripetiamo l'apprendimento ma eseguiamo prima il data augmentation

In [ ]:
# agiungiamo (un po' causalmente) dati eseguendo rotazioni, zoom e ribaltamenti
MM=ImageDataGenerator(rescale=1/255., validation_split=0.2,horizontal_flip=True,rotation_range=0.05,width_shift_range=0.01,zoom_range=0.1)
datatrain= MM.flow_from_directory('JPG',target_size=(299,299),subset='training',classes=classe,class_mode='categorical',batch_size=32,shuffle=True)
dataval= MM.flow_from_directory('JPG',target_size=(299,299),subset='validation',classes=classe,class_mode='categorical',batch_size=32)

Found 256 images belonging to 4 classes.
Found 64 images belonging to 4 classes.


In [ ]:
#riutilizziamo lo stesso modello

inp=Input(shape=(299,299,3))
#scegliamo un kernel 5 X 5 
x=Conv2D(64,(5,5),activation='relu')(inp)
# nel secondo layer di convoluzione diminuiamo il numero di parametri a 32
x=Conv2D(32,(5,5),activation='relu')(x)
x=MaxPooling2D()(x)
x=Conv2D(32,(3,3),activation='relu')(x)
x=MaxPooling2D()(x)
x=Conv2D(16,(3,3),activation='relu')(x)
x=MaxPooling2D()(x)
x=Flatten()(x)
x=Dense(64,activation='relu')(x)
x=Dropout(0.3)(x)
out=Dense(len(vocab),activation='softmax')(x)
model=Model(inp,out)
model.summary()

Model: "model_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_7 (InputLayer)        [(None, 299, 299, 3)]     0         
                                                                 
 conv2d_13 (Conv2D)          (None, 295, 295, 64)      4864      
                                                                 
 conv2d_14 (Conv2D)          (None, 291, 291, 32)      51232     
                                                                 
 max_pooling2d_13 (MaxPoolin  (None, 145, 145, 32)     0         
 g2D)                                                            
                                                                 
 conv2d_15 (Conv2D)          (None, 143, 143, 32)      9248      
                                                                 
 max_pooling2d_14 (MaxPoolin  (None, 71, 71, 32)       0         
 g2D)                                                      

In [ ]:
# il fitting viene fatto girare piu' volte finche' no otteniamo un valore di 
# accuratezza abbastanza alto

model.compile(loss='categorical_crossentropy',metrics=['acc'])
#come training data e' sufficiente specificare l'iteratore datatrain
model.fit(datatrain,epochs=10,validation_data=dataval)

Epoch 1/10
8/8 [==============================] - 9s 1s/step - loss: 1.3467 - acc: 0.7461 - val_loss: 0.3581 - val_acc: 0.8438
Epoch 2/10
8/8 [==============================] - 8s 984ms/step - loss: 0.3962 - acc: 0.8477 - val_loss: 0.2904 - val_acc: 0.8906
Epoch 3/10
8/8 [==============================] - 8s 993ms/step - loss: 0.3469 - acc: 0.8828 - val_loss: 0.2430 - val_acc: 0.8906
Epoch 4/10
8/8 [==============================] - 8s 969ms/step - loss: 0.4712 - acc: 0.8320 - val_loss: 0.2299 - val_acc: 0.9219
Epoch 5/10
8/8 [==============================] - 8s 964ms/step - loss: 0.4111 - acc: 0.8789 - val_loss: 0.1996 - val_acc: 0.9531
Epoch 6/10
8/8 [==============================] - 8s 941ms/step - loss: 0.3282 - acc: 0.8633 - val_loss: 0.1733 - val_acc: 0.9219
Epoch 7/10
8/8 [==============================] - 7s 934ms/step - loss: 0.4453 - acc: 0.8438 - val_loss: 0.2077 - val_acc: 0.9062
Epoch 8/10
8/8 [==============================] - 7s 935ms/step - loss: 0.8227 - acc: 0.7734 

Quando si fa data augmentation, e' possibile ottenere un'accuratezza maggiore nel validation set rispetto al training set. Questo avviene perche' stiamo aggiungendo dati sempre nuovi al training e questo puo' diminuire l'accuratezza, mentre il validation set resta sempre fisso.

In molti casi e' possibile importare reti che hanno gia' fatto il training su grandi dataset di immagini e ri-adattarle alle nostre esigenze. Keras mette a disposizione molte di queste reti. In questo caso useremo il modello InceptionV3.

Il metodo da seguire e' quello di prendere la rete, "congelare" tutti i layers tranne gli ultimi due, e fare il training solo su quelli. Una volta che abbiamo fatto quel training, possiamo scongelare l'intera rete e fare un altro training. Questa tecnica permette di imparare piu' rapidamente, visto che il grosso del nuovo apprendimento dovra' essere fatto sugli ultimi layer (che inizialmente davano l'output per una classificazione su 1000 classi, invece delle nostre quattro).

In [ ]:
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.layers import GlobalAveragePooling2D
# gli iltimi due strati della rete non verranno importati (sono quelli che 
# dobbiamo adattare al nostro problema specifico)
base_model=InceptionV3(weights='imagenet',include_top=False)

87924736/87910968 [==============================] - 1s 0us/step


In [ ]:
base_model.summary()

Model: "inception_v3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_8 (InputLayer)           [(None, None, None,  0           []                               
                                 3)]                                                              
                                                                                                  
 conv2d_17 (Conv2D)             (None, None, None,   864         ['input_8[0][0]']                
                                32)                                                               
                                                                                                  
 batch_normalization (BatchNorm  (None, None, None,   96         ['conv2d_17[0][0]']              
 alization)                     32)                                                    

In [ ]:
x=base_model.output
# la funzione GlobalAveragePooling restituisce la media dei valori quando viene 
# fatto il pooling
x=GlobalAveragePooling2D()(x)
x=Dropout(0.3)(x)
x=Dense(512,activation='relu')(x)
out=Dense(len(vocab),activation='softmax')(x)
# come input usiamo quello del base model
model=Model(base_model.input,out)

In [ ]:
# prima di iniziare il training dobbiamo congelare i pesi
for layer in base_model.layers:
  layer.trainable=False

In [ ]:
model.compile(loss='categorical_crossentropy',metrics=['acc'])
model.fit(datatrain,epochs=10,validation_data=dataval)

Epoch 1/10
8/8 [==============================] - 14s 1s/step - loss: 2.7503 - acc: 0.5273 - val_loss: 0.1849 - val_acc: 0.9688
Epoch 2/10
8/8 [==============================] - 7s 883ms/step - loss: 0.1792 - acc: 0.9414 - val_loss: 0.1042 - val_acc: 1.0000
Epoch 3/10
8/8 [==============================] - 7s 885ms/step - loss: 0.4442 - acc: 0.8906 - val_loss: 2.4656 - val_acc: 0.5938
Epoch 4/10
8/8 [==============================] - 7s 886ms/step - loss: 0.3696 - acc: 0.9102 - val_loss: 0.0354 - val_acc: 0.9844
Epoch 5/10
8/8 [==============================] - 7s 888ms/step - loss: 0.0575 - acc: 0.9844 - val_loss: 0.0432 - val_acc: 0.9844
Epoch 6/10
8/8 [==============================] - 7s 887ms/step - loss: 0.0517 - acc: 0.9883 - val_loss: 0.1277 - val_acc: 0.9531
Epoch 7/10
8/8 [==============================] - 7s 888ms/step - loss: 0.0717 - acc: 0.9727 - val_loss: 1.0891 - val_acc: 0.7344
Epoch 8/10
8/8 [==============================] - 7s 881ms/step - loss: 0.7325 - acc: 0.8203

Se la loss function non diminuisce monotonicamente probabilmente e' necessario cambiare qualcosa nel set up. Nel nostro caso probabilmente siamo stati troppo estremi nell'aggiunta di immagini nel data augmentation.

Una volta fatto il training degli ultimi due layers, ne scongeliamo degli altri e alleniamo quelli. Stampare i layer e le funzioni del base-model ci aiuta a decidere quali layer allenare.

In [ ]:
i=0
for 

In [ ]:
for layer in base_model.layers[:249]:
  layer.trainable=False

for layer in base_model.layers[249:]:
  layer.trainable=True